In [1]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D,MaxPooling2D,ZeroPadding2D
from keras.preprocessing import image
import numpy as np

Using TensorFlow backend.


In [3]:
def load_image():
    T = []
    for i in range(1, 8001):
        if i < 10:
            location = '000' + str(i)
        elif i < 100:
            location = '00' + str(i)
        elif i < 1000:
            location = '0' + str(i)
        else:
            location = str(i)
        image_load = image.load_img('./solved_data/0' + location + '.jpg', target_size = (64, 64))
        image_load = image.img_to_array(image_load)
        T.append(image_load)
    return T
x = load_image()
#data #8000*64*64*3

In [4]:
x = np.array(x)

In [5]:
x.shape

(8000, 64, 64, 3)

In [6]:
import pandas as pd
train_dataset = pd.read_csv('annotations.csv')
target = train_dataset['class']

In [7]:
y = []
for i in range(0, 8000):
    y.append(int(target[i]))

In [8]:
y = np.array(y)

In [9]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.1)

In [10]:
x_train_nomalized = x_train.astype("float")/255
x_test_normalized = x_test.astype("float")/255

In [11]:
from keras.utils import np_utils
y_train_onehot = np_utils.to_categorical(y_train)
y_test_onehot = np_utils.to_categorical(y_test)

In [12]:
model = Sequential()
model.add(Conv2D(filters = 64, kernel_size = (2, 2), input_shape = (64, 64, 3), activation = 'relu'))
model.add(Dropout(rate=0.1))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Conv2D(filters = 64, kernel_size = (3, 3), activation = 'relu'))
model.add(Dropout(0.1))
model.add(MaxPooling2D(pool_size = (2,2)))

model.add(Flatten()) #把多维的输入一维化

###从卷积层到全连接层的过渡###
model.add(Dense(1000, activation = 'relu'))
model.add(Dense(500, activation = 'sigmoid'))
model.add(Dense(197, activation = 'softmax'))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [13]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 63, 63, 64)        832       
_________________________________________________________________
dropout_1 (Dropout)          (None, 63, 63, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 31, 31, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 29, 29, 64)        36928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 29, 29, 64)        0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 14, 14, 64)        0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 12544)             0         
__________

In [14]:
#定义参数
epochs = 10
batch_size = 50

In [15]:
model.compile(loss = 'categorical_crossentropy',
             optimizer = 'adam', metrics = ['accuracy'])

train_history = model.fit(x_train_nomalized, y_train_onehot,
                         validation_split = 0.1,
                         epochs = epochs, batch_size = batch_size)

Instructions for updating:
Use tf.cast instead.
Train on 6480 samples, validate on 720 samples
Epoch 1/10
6480/6480 [==============================] - 22s 3ms/step - loss: 5.3296 - acc: 0.0057 - val_loss: 5.2142 - val_acc: 0.0097
Epoch 2/10
6480/6480 [==============================] - 10s 2ms/step - loss: 5.1798 - acc: 0.0088 - val_loss: 5.1297 - val_acc: 0.0153
Epoch 3/10
6480/6480 [==============================] - 10s 2ms/step - loss: 5.0588 - acc: 0.0177 - val_loss: 5.0357 - val_acc: 0.0181
Epoch 4/10
6480/6480 [==============================] - 10s 2ms/step - loss: 4.7778 - acc: 0.0415 - val_loss: 4.7169 - val_acc: 0.0556
Epoch 5/10
6480/6480 [==============================] - 10s 2ms/step - loss: 4.2255 - acc: 0.1071 - val_loss: 4.4683 - val_acc: 0.0639
Epoch 6/10
6480/6480 [==============================] - 10s 2ms/step - loss: 3.4139 - acc: 0.2386 - val_loss: 4.3384 - val_acc: 0.0972
Epoch 7/10
6480/6480 [==============================] - 10s 2ms/step - loss: 2.4480 - acc: 0.45

In [15]:
score = model.evaluate(x_test_normalized, y_test_onehot, batch_size = 50)

800/800 [==============================] - 0s 321us/step


In [16]:
print("test accuracy:{}%".format(int(score[1] * 10000)/100))

test accuracy:13.12%
